[Google Drive link to these ZIP files](https://drive.google.com/drive/folders/1W2GPNJ3g5zkHQ3RAaz_S7-arONyKgCNR?usp=sharing)

### Create count2-pruned

In [ ]:
from counter import counter

In [ ]:
cc = counter()

In [ ]:
cc.count(    
    jstor_zip_base = "C:/Users/benja/OneDrive - Cornell University/Desktop/soc_econ/data",
    
    SKIP_N = 10*5 - 1,
    #MAX_N = 100,
    combinations = [
        ('fy','t'), # from year, term
        ('fj','t'),
        #('fj','fy','t'),
        ('fy',), # occurences at all
        ('t',),
        ('fj',),
    ]
    
)

In [ ]:
cc(t='social')

In [ ]:
# prune a bit before dumping it to disk
to_delete = [t for t in cc.items('t') if cc(t=t) < 50]

cc.delete('t', to_delete)
cc.prune_zeros()

In [ ]:
cc(t='social')

In [ ]:
cc.save_counts('count2-pruned')

### setting date range

In [ ]:
cc = counter('count2-pruned')

In [ ]:
mny = min(cc.items('fy'))
mxy = max(cc.items('fy'))

In [ ]:
ys = range(mny, mxy+1)
cs = [cc(fy=y) for y in ys]

plt.plot(ys,cs);

### first filter based on number of times each term was mentioned

We want this to be as wide a net we can stand for the next filtering processes.
I cannot keep track of yearly counts of all terms, or even simple term counts (at least, not without dynamic consolidation), because of RAM limitations.
Remember that I'm keeping track of all words, but also all two-word tuples!
So this first stage is only using 1/10 of the documents (see above, SKIP_N).

<!--At this point, subsetting the data now that we have a terms list will not help our memory issue.-->
The term-term coocurrence network uses N^2 integers, which with 1M terms is 1M^2 = 1TB of memory.
If we can limit to ~30,000 terms, approximately the working vocabulary of an adult, this would fit in 1GB of memory.
We can filter by term *dynamics* instead of just relying on .
We can also limit the 1-tuples and 2-tuples independently, reserving 15,000 terms for each.
2-tuples currently make up the majority of our counts, but are not inherently more important than 1-tuples.

In [ ]:
#plt.plot([t._dp[y] for y in range(1950,2015)])

In [ ]:
target_N = 5000

In [ ]:
terms = cc.items('t')
counts = np.array([ cc(t=t) for t in terms ])
cdict = {t:cc(t=t) for t in terms}

In [ ]:
plt.hist(counts[counts<500], bins=50, log=True);

In [ ]:
tup1 = [x for x in terms if '-' not in x]
tup2 = [x for x in terms if '-' in x]

In [ ]:
tup1_top = sorted(tup1, key=lambda x:cdict[x])[-target_N//2: ]
tup2_top = sorted(tup2, key=lambda x:cdict[x])[-target_N//2: ]

In [ ]:
terms_to_keep = tup1_top + tup2_top

In [ ]:
import pickle

with open('terms2.pickle', 'wb') as outf:
    pickle.dump(terms_to_keep, outf)

In [ ]:
sample( terms_to_keep, 10 )

### recount with fuller detail

on my desktop computer this takes way too much time

In [22]:
from counter import counter

In [23]:
import pickle

with open('terms.pickle', 'rb') as inf:
    terms_to_keep = pickle.load(inf)

In [24]:
c = counter()

Blank counter with no name


In [25]:
c.count(    
    jstor_zip_base = "C:/Users/benja/OneDrive - Cornell University/GitHub/kk-jstor-sociology",

    SKIP_N = 50 - 1,
    #MAX_N = 100*100,
    combinations = [
        #('fy','t'),
        #('fj','t'),
        #('fj','fy','t'),
        ('t1', 't2'),
        ('t1', 't2', 'fy'),
        ('fy',),
        ('t',),
        #('fj',),
    ],

    term_whitelist = terms_to_keep,
    WINDOW=250

)

Will print updated statistics every 20 documents.
Found files 
Iterating over  0 documents


In [26]:
c.counts[('t',)].sum()

KeyError: ('t',)

In [27]:
c.save_counts('count-cooc-2pct-250')

In [28]:
c2 = counter()

Blank counter with no name


In [ ]:
c2.count(    
    jstor_zip_base = "G:/My Drive/2020 ORGANISATION/1. PROJECTS/qualitative analysis of literature/110 CITATION ANALYSIS/000 data/sociology jstor",

    SKIP_N = 50 - 1,
    #MAX_N = 100*100,
    combinations = [
        #('fy','t'),
        #('fj','t'),
        #('fj','fy','t'),
        ('t1', 't2'),
        ('t1', 't2', 'fy'),
        ('fy',),
        ('t',),
        #('fj',),
    ],

    term_whitelist = terms_to_keep,
    WINDOW=25

)

In [ ]:
c.save_counts('count-cooc-2pct-25')